## 1. Installations


In [6]:
!/usr/local/bin/python3 -m pip install -q --upgrade duckdb


In [7]:
!pip3 install --upgrade pip

In [9]:
pip install kagglehub[pandas-datasets] python-dotenv


zsh:1: no matches found: kagglehub[pandas-datasets]
Note: you may need to restart the kernel to use updated packages.


## 2. Imports


In [39]:
import os
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import kagglehub
from kagglehub import KaggleDatasetAdapter


## 3. Downloads from Kaggle


In [18]:
load_dotenv()
print("USER:", os.getenv("KAGGLE_USERNAME"))
print("KEY:", "OK" if os.getenv("KAGGLE_KEY") else "MISSING")


USER: ananobre
KEY: OK


In [26]:
# Download latest version
path = kagglehub.dataset_download("computingvictor/transactions-fraud-datasets")

print("Path to dataset files:", path)

Path to dataset files: /Users/ananobre/.cache/kagglehub/datasets/computingvictor/transactions-fraud-datasets/versions/1


In [38]:
folder = "/Users/ananobre/.cache/kagglehub/datasets/computingvictor/transactions-fraud-datasets/versions/1"
print(os.listdir(folder))



['train_fraud_labels.json', 'mcc_codes.json', 'users_data.csv', 'transactions_data.csv', 'cards_data.csv']


## 4. Load Data


### 🔎 users_data.csv

In [ ]:
# CSVs
users = pd.read_csv(os.path.join(folder, "users_data.csv"))
transactions = pd.read_csv(os.path.join(folder, "transactions_data.csv"))
cards = pd.read_csv(os.path.join(folder, "cards_data.csv"))

# Checagem rápida
print("users:", users.shape, "| transactions:", transactions.shape, "| cards:", cards.shape)

users.head()


users: (2000, 14) | transactions: (13305915, 12) | cards: (6146, 13)


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


### 🔎 transactions_data.csv"


In [44]:
transactions.head()


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN



### 🔎 cards_data.csv"

In [45]:
cards.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [46]:
# JSONs (dict simples -> vira duas colunas)
with open(os.path.join(folder, "mcc_codes.json"), "r", encoding="utf-8") as f:
    mcc_codes = pd.DataFrame(list(json.load(f).items()), columns=["mcc", "mcc_description"])

with open(os.path.join(folder, "train_fraud_labels.json"), "r", encoding="utf-8") as f:
    fraud_labels = pd.DataFrame(list(json.load(f).items()), columns=["transaction_id", "is_fraud"])

# Checagem rápida
print("mcc_codes:", mcc_codes.shape, "| fraud_labels:", fraud_labels.shape)

mcc_codes.head()



mcc_codes: (109, 2) | fraud_labels: (1, 2)


,mcc,mcc_description
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


In [47]:
fraud_labels.head()

,transaction_id,is_fraud
0,target,"{'10649266': 'No', '23410063': 'No', '9316588'..."


## 5. Exploratory Data Analysis (EDA)


In [49]:
for name, df in {
    "users": users, 
    "transactions": transactions, 
    "cards": cards, 
    "fraud_labels": fraud_labels, 
    "mcc_codes": mcc_codes
}.items():
    print(f"\n{name.upper()} -> {df.shape}")
    print(df.dtypes)
    print(df.head(3))



USERS -> (2000, 14)
id                     int64
current_age            int64
retirement_age         int64
birth_year             int64
birth_month            int64
gender                object
address               object
latitude             float64
longitude            float64
per_capita_income     object
yearly_income         object
total_debt            object
credit_score           int64
num_credit_cards       int64
dtype: object
     id  current_age  retirement_age  birth_year  birth_month  gender  \
0   825           53              66        1966           11  Female   
1  1746           53              68        1966           12  Female   
2  1718           81              67        1938           11  Female   

                  address  latitude  longitude per_capita_income  \
0           462 Rose Lane     34.15    -117.76            $29278   
1  3606 Federal Boulevard     40.76     -73.74            $37891   
2         766 Third Drive     34.02    -117.89            $226

In [50]:
users.columns


Index(['id', 'current_age', 'retirement_age', 'birth_year', 'birth_month',
       'gender', 'address', 'latitude', 'longitude', 'per_capita_income',
       'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards'],
      dtype='object')

In [51]:
transactions.columns


Index(['id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors'],
      dtype='object')

In [52]:
cards.columns


Index(['id', 'client_id', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'has_chip', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web'],
      dtype='object')

In [53]:
fraud_labels.columns


Index(['transaction_id', 'is_fraud'], dtype='object')

In [54]:
mcc_codes.columns


Index(['mcc', 'mcc_description'], dtype='object')

In [ ]:
df = (
    transactions
    .merge(users, left_on="client_id", right_on="id", how="left", suffixes=("", "_user"))
    .merge(cards, left_on="card_id", right_on="id", how="left", suffixes=("", "_card"))
    .merge(fraud_labels, left_on="id", right_on="transaction_id", how="left")
    .merge(mcc_codes, on="mcc", how="left")
)
print(df.shape)
df.head()
